In [3]:
!pip install gradio -U -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.3 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.3 which is incompatible.
notebook 6.5.5 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.3.1 which is incompatible.
notebook 6.5.5 requires pyzmq<25,>=17, but you have pyzmq 25.1.1 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.2 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.15.0 which is incompatible.


In [28]:
import re
import random
import gradio as gr
import requests
import json

# Global
controller_url = "ip-172-31-70-124.ap-southeast-2.compute.internal" + ":8000"

l_mapping = {
    "Bash":"shell", 
    "Python":"python", 
    "Java":"javascript",
    "JavaScript":"javascript", 
    "R":"r"
}
models_list = {
    "Claude-v2": {
        "model_family": "bedrock",
        "model_name": "anthropic.claude-v2"
    }
}

css = """
#red {background-color: #FA9F9D}
#amber {background-color: #FFD966}
.gradio-container {margin: 0 !important}
.app.svelte-1kyws56.svelte-1kyws56 {
  max-width: 2100px;
}
"""
    
def execute_fn(state, history, script, exp_out, model, language):
    data = json.dumps(
        {
            "script": script,
            "model_family": models_list[model]["model_family"], 
            "model_name": models_list[model]["model_name"], 
            "language": language,
            "expected_output": exp_out,
            "conv_id": state
        }
    )
    response = requests.post(
            "http://" + controller_url + "/execute",
            data=data
        ).json()
    if response["error"]:
        out = gr.Textbox(value=response["output"],label="Output: ❌️ EERROR ❌️", elem_id="red")
        history.append(["The script failed with shown error message", response["generated_text"]])
        script = response["script"]
    else:
        if response["output"] != exp_out:
            out = gr.Textbox(value=response["output"],label="Output: ⚠️ WARNING ⚠️ value does not match `Expected Output`", elem_id="amber", interactive=False, show_copy_button=True, lines=7)
        else:
            out = gr.Textbox(value=response["output"],label="Output:", elem_id="--primary-50", interactive=False, show_copy_button=True, lines=7)
    return [history, out, script] 


def can_exec(script):
    if script != "":
        return gr.Button(value="Approve and Execute", interactive=True)
    else:
        return gr.Button(value="Approve and Execute", interactive=False)

    
def Dropdown_lang(dropdown, script):
    """
    Code Languages
    approved language: [('python', 'markdown', 'json', 'html', 'css', 'javascript', 'typescript', 'yaml', 'dockerfile', 'shell', 'r')]
    """
    return [gr.Code(value=script, language=l_mapping[dropdown], interactive=False), ""]

def clear_fn():
    return [""] * 3 + [gr.Textbox(value="",label="Output", elem_id="--primary-50", interactive=False, show_copy_button=True, lines=7)]

def pass_chat(message, history):
    return ""

def generate_response(state, message, history, model, language): 
    """
    Sample Response - to be deleted
    """
    data = json.dumps(
        {
            "prompt": message,
            "model_family": models_list[model]["model_family"], 
            "model_name": models_list[model]["model_name"], 
            "language": language,
            "conv_id": state
        }
    )
    response = requests.post(
            "http://" + controller_url + "/generate",
            data=data
        ).json()
    state = response["conv_id"]
    history += [[message, response["generated_text"]]]
    return [state, history, response["script"], response["expected_output"]]

def web_ui():
    with gr.Blocks(css=css, theme=gr.themes.Base(neutral_hue="slate")) as webUI:
        state = gr.State(value="")
        gr.Markdown(
            """
            # Welcome to SynthCodeX 🤖️
            ### Use this tool to generate and test code using LLMs
            """)
        with gr.Row(equal_height=True):
            with gr.Column(scale=2):
                chat = gr.ChatInterface(
                    pass_chat, 
                    undo_btn=None, 
                    retry_btn=None
                )
            with gr.Column(scale=1):
                with gr.Group():
                    language = gr.Dropdown(languages,label='Langauge Selection', value=languages[0]) # Langauge Selection
                    model = gr.Dropdown(models_list.keys(),label="Model Selection", value=list(models_list.keys())[0]) # Model Selection
                    script = gr.Code(value="",label="Script", language = l_mapping[languages[0]], interactive=False)
                    exp_out = gr.Textbox(value="",label="Expected Output", interactive=False)
                    execute = gr.Button(value="Approve and Execute", interactive=False)
        out = gr.Textbox(value="",label="Output", interactive=False, show_copy_button=True, lines=7)
        with gr.Accordion(label="Parameters", open=False):
            auto_run = gr.Checkbox(label='Auto-approve and Execute')
            timeout = gr.Number(label="Timeout", precision=0, minimum=0, maximum=60)
            
        language.change(Dropdown_lang, [language, script], [script, out])
        script.change(can_exec, script, execute)
        chat.submit_btn.click(generate_response, [state, chat.textbox, chat.chatbot, model, language], [state, chat.chatbot, script, exp_out])
        chat.textbox.submit(generate_response, [state, chat.textbox, chat.chatbot, model, language], [state, chat.chatbot, script, exp_out])
        chat.clear_btn.click(clear_fn, None, [state, script, exp_out, out])
        execute.click(execute_fn, [state, chat.chatbot, script, exp_out, model, language], [chat.chatbot, out, script])
    return webUI

def get_languages_list(url):
    return list(json.loads(requests.get("http://" + url + "/list_languages").json()).keys())
    
if __name__ == "__main__":    
    languages = get_languages_list(controller_url)
    UI = web_ui()
    UI.launch(debug=True)

    # create events, when chaning model/language selection, it will clear the chat history, scirpt, expected output and output


Running on local URL:  http://127.0.0.1:7861

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Sagemaker notebooks may require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://61ab7f50e0c7ce9cb9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://0516bcd1d5391d01e7.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://61ab7f50e0c7ce9cb9.gradio.live


# Reset Sample

In [ ]:
import gradio as gr

def converse(x, y, z):
    return z

def reset(z):
    return [], []

# Gradio app
with gr.Blocks() as demo:
    bot = gr.Chatbot(render=False)
    dropdown = gr.Dropdown(["Character 1", "Character 2", "Character 3"], label="Characters", info="Select the character that you'd like to speak to", value="Character 1")
    chat = gr.ChatInterface(
        fn=converse,
        chatbot=bot,
        additional_inputs=dropdown
    )
    dropdown.input(fn=reset, inputs=dropdown, outputs=[bot, chat.chatbot_state])

demo.queue()
demo.launch()

In [44]:
"http://" + controller_url + "/generate"

'http://ip-172-31-73-215.ap-southeast-2.compute.internal:8000/generate'